# Лекция 3. Регулярные выражения

> __Регулярные выражения__ - это мощный способ поиска и замены для строк по достаточно сложным условиям. Фактически являются специальным языком шаблонов.

С простыми шаблонами мы уже знакомы - проверки на содержимое строки делались довольно часто

```
if s == "hello":
  pass
```

Хотелось бы иметь инструмент, который позволит проводить анализ строки более гибко. Этим инструментом являются _регулярные выражения_. Каждый язык реализует свою библиотеку по работе с этим языком шаблонов. В Python - это сделано в модуле `re`.

In [1]:
import re

Регулярные выражения обычно решают следующие задачи:
- `re.match(pattern, string, flags=0)` проверка строки на совпадение с шаблоном
- `re.search(pattern, string, flags=0)`, `re.findall(pattern, string, flags)` (поиск подстроки (подстрок) в строке согласно шаблону
- ` re.sub(pattern, repl, string, count=0, flags=0)`, `re.subn(pattern, repl, string, count=0, flags=0)` замена подстрок в строке согласно шаблону
- `re.split(pattern, string, maxsplit=0, flags=0)` разделение строки на подстроки с помощью шаблона (сложный аналог split)

# Метасимволы

Следующие символы являются специальными и обрабатываются особенным образом
```
. ^ $ * + ? { } [ ] \ | ( )
```

Если необходимо обработать данные символы в виде обычных символов, мы можем использовать экранировку - просто добавить символ `\` перед нужным символом. Например `\^`, `\{`.

Все остальные символы внутри шаблона воспринимаются как есть. Если написано `a`, то это означает, что требуется совпадение с символом `a`.

# Набор

Метасимволы `[` и `]` позволяют задать набор символов (класс символа), под которые должен подходить __один__ символ строки. Для этого внутри скобок перечисляются подходящие символы. Если `-` используется как символ, а не диапазон, то он должен указываться последним. Также можно указывать диапазон (согласно таблице кодировки). Метасимфолы внутри скобок не работают.

Например: 
- `[0-9]` - число от 0 до 9
- `[uaw]` - один символ, либо u, либо a, либо w
- `[0-9a-zA-Z-]` - буквы, цифры и знак дефиса

In [6]:
# [uaw] эквивалентно следующему коду

char = 'a'
result = char in ['u', 'a', 'w']

Если добавить в начале символ `^`, то это инвертирует условие

Например: 
- `[^0-9]` - все что угодно, кроме чисел от 0 до 9
- `[^uaw]` - любой символ, кроме u, a и w
- `[^0-9a-zA-Z-]` - любой символ, кроме буквы, цифры и знака дефиса

In [4]:
# [^uaw] эквивалентно следующему коду

char = 'a'
result = char not in ['u', 'a', 'w']

Помимо этого, существуют алиасы наиболее популярных наборов (это не все)

- `\d` - эквивалентно `[0-9]`
- `\D` - эквивалентно `[^0-9]`
- `\s` - эквивалентно `[ \t\n\r\f\v]`
- `\S` - эквивалентно `[^ \t\n\r\f\v]`
- `\w` - эквивалентно `[a-zA-Z0-9_]`
- `\W` - эквивалентно `[^a-zA-Z0-9_]`

Алиасы можно использовать по отдельности, так и внутри `[]`.
- `[\s\w]` - любая буква, цифра, символ _ и пробельные символы
- `\d`, `[\d]` и `[0-9]` - это одно и тоже обозначение цифры

In [102]:
# match возвращает объект Match(информация об найденном), если что-то нашли и None, если ничего не найдено

print(re.match(r'[\d]', "1") is not None)
print(re.match(r'\d', "1") is not None)
print(re.match(r'[0-9]', "1") is not None)
print(re.match(r'\d', "a") is not None)

True
True
True
False


Здесь выше используется литерал "сырой"(raw) строки. В этой строки все символы воспринимаются как есть (`\n` - это два символа, а не символ переноса строки). Без этого пришлось бы писать так `[\\d]`

# Джокер

Метасимвол `.` обозначает просто любой символ, за исключением символа переноса строки. Если включить режим `re.DOTALL`, то включает и его.

# Комбинирование

Все выше описанное можно использовать для описания совокупности символов внутри строки
- `\d\d` - две любых цифры
- `[ab][cd]` - строки ac, ad, bc, bd
- `.a` - все строки из двух символов, оканчивающиеся на `a`
- `...` - любая строка из трех символов
- `[aA]..` - любоая строка из трех символов, начинающая с `a` или `A`

# Квантификаторы

Все выше, конечно, удобно, но что делать если не всегда известно сколько символов в слове или строке? Или стоит задача найти все слова начинающиеся с буквы `A`?

На помощь приходят __квантификаторы__, они определяют сколько раз должны повторятся стоящие перед ними символ, класс или группа.

- `{n}` - повторяется ровно n раз
- `{n,m}` - повторяется от n до m раз
- `{n,}` - повторяется не менее n раз
- `{,m}` - повторяется не более m раз
- `?` или `{0,1}` - повторяется один и менее раз
- `*` или `{0,}` - повторяется от нуля до любого количества раз
- `+` или `{1,}` - повторяется от одного раза до любого количества

Примеры
- `aaa` или `a{3}` - совпадает со сторокой `aaa`
- `[ab]{2}` - строки aa, ab, ba, bb
- `.*` - строка из любого количества любых символов
- `a+` - строки `a`, `aa`, `aaa`, ...
- `[aA].*` - строка начинающаяся с буквы `a`

Важно отметить, что квантификаторы выше - "жадные". Они пытаются откусить максимально возможный кусок от строки.

In [105]:
m = re.match("a+", "aaaaaaaabbbb")
m.group()

'aaaaaaaa'

Чтобы изменить поведение с "жадного" на "ленивое" (минимально возможная строка) нужно добавить к квантификатору знак `?`

- `+?`
- `*?`
- `{n,m}?`

In [107]:
s = '<span class="title">Hello World</span>'

m = re.search(r'<.*>', s)
print(m[0])

m = re.search(r'<.*?>', s)
print(m[0])

<span class="title">Hello World</span>
<span class="title">


# Или

Имеется возможность добавить альтернативный шаблоны с помощью метасимвола `|`

- `asd|zxc|qwe` - либо строка asd, либо строка zxc
- `Hello|[aA].*` - либо Hello, либо любое слово на a

# Группы

С помощью метасимволов `(` и `)` можно выделять группы внутри шаблона. Их можно использовать для извлечения информации, так и для дополнительного уточнения шаблона. Группы нумеруются по порядку счета открывающих скобок слева направо. У самого шаблона нулевая группа.

- `(asd)(zxc)` - строка asdzxc, разбитая на две группы
- `(Hello|Hi) World!` - строки `Hello World!` и `Hi World!`
- `(Hello)+` - строки `Hello`, `HelloHello`, `HelloHelloHello`, ...

In [108]:
m = re.search(r"(ab)+", "abababababab")
print(m.group(0), m.group(1))

abababababab ab


# Начало и конец

- `^` - данный символ обозначает начало строки (меняет свое поведение в зависимости от флага `re.MULTILINE`)
- `$` - обозначает конец строки (меняет свое поведение в зависимости от флага `re.MULTILINE`)

In [110]:
text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. 
Pellentesque et mi ultrices neque elementum dignissim a ac dolor. 
Nulla at orci non purus mollis finibus. 
Donec interdum tellus et urna efficitur, vitae placerat lorem pellentesque. 
Donec lacinia mi in lacinia tristique. 
Suspendisse turpis risus, faucibus vel lacus vel, viverra pulvinar eros. 
Duis id elit et mauris tincidunt placerat. 
Cras mollis id mi vitae pulvinar. 
Duis at quam nec nunc pretium dapibus sed eget eros. 
Maecenas ultrices porttitor enim id viverra. 
Praesent imperdiet orci ullamcorper arcu malesuada, vel consectetur mi sagittis. 
Suspendisse mollis aliquet metus vitae egestas. 
Donec eget quam tempus, faucibus leo pulvinar, lacinia dui. 
Fusce in metus eros. 
Donec mollis velit eget tortor commodo, eu sollicitudin mi pharetra. 
Vestibulum fermentum dui in velit egestas tincidunt. 
Donec laoreet luctus rhoncus. 
"""

print(re.findall(r'^D.*', text))
print(re.findall(r'^d.*', text, re.MULTILINE|re.IGNORECASE))

[]
['Donec interdum tellus et urna efficitur, vitae placerat lorem pellentesque. ', 'Donec lacinia mi in lacinia tristique. ', 'Duis id elit et mauris tincidunt placerat. ', 'Duis at quam nec nunc pretium dapibus sed eget eros. ', 'Donec eget quam tempus, faucibus leo pulvinar, lacinia dui. ', 'Donec mollis velit eget tortor commodo, eu sollicitudin mi pharetra. ', 'Donec laoreet luctus rhoncus. ']


In [111]:
print(re.findall(r'.*I\.\s?$', text))
print(re.findall(r'.*I\.\s?$', text, re.MULTILINE|re.IGNORECASE))

[]
['Donec eget quam tempus, faucibus leo pulvinar, lacinia dui. ']


In [43]:
bin(re.MULTILINE), bin(re.IGNORECASE)

('0b1000', '0b10')

# Обратная связь

Группы удобно использовать при заменах, значения пойманное в группу можно использовать повторно.
- `\g<1>` - первая группа
- `\1` - первая группа

In [112]:
# Компилируем выражение (положительно влияет на скорость, если использовать часто)
rx = re.compile(r'Hello\s+(.+?)!')

s = "lfdsgf kadjfg lsdkjg sdg.j as.djkg. Hello World! sal asd jghkw gh"

print(rx.sub(r"Hi \1!", s))
print(rx.sub(r"Hi \g<1>!", s))

lfdsgf kadjfg lsdkjg sdg.j as.djkg. Hi World! sal asd jghkw gh
lfdsgf kadjfg lsdkjg sdg.j as.djkg. Hi World! sal asd jghkw gh


# Пример

Разбор даты в ISO

In [114]:
date1 = '2007-03-01T13:00:00Z'
date2 = '2007-03-01T13:00:00+03:30'
date3 = '2005-08-09T18:31:42.201'


rx = re.compile(
    r'^(\d{4})-(\d{2})-(\d\d)T(\d{2}):(\d{2}):(\d{2})(\.[0-9]{3})?(Z|[+-]\d\d:\d\d)?$'
)

In [115]:
m = rx.search(date1)
m.groups()

('2007', '03', '01', '13', '00', '00', None, 'Z')

In [93]:
m = rx.search(date2)
m.groups()

('2007', '03', '01', '13', '00', '00', None, '+03:30')

In [94]:
m = rx.search(date3)
m.groups()

('2005', '08', '09', '18', '31', '42', '.201', None)